In [0]:
!pip install -qU langchain langchain-openai langchain-community langchain-huggingface langchain-openai transformers spacy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.10.3 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
pip install --upgrade diffusers[torch]

  Obtaining dependency information for diffusers[torch] from https://files.pythonhosted.org/packages/07/7f/53828ff41b86c3274099327627ff5cb7581a2cc9db3bcb39e5212c4140b7/diffusers-0.31.0-py3-none-any.whl.metadata
  Using cached diffusers-0.31.0-py3-none-any.whl.metadata (18 kB)
Using cached diffusers-0.31.0-py3-none-any.whl (2.9 MB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
dbutils.library.restartPython()

In [0]:
import asyncio
import spacy
from getpass import getpass
import os

from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, create_openai_tools_agent
from langchain_huggingface import HuggingFacePipeline
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from transformers import pipeline

2024-12-14 12:45:36.604932: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-14 12:45:36.609845: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-14 12:45:36.674087: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-14 12:45:38.102012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [0]:
# HUGGINGFACEHUB_API_TOKEN = getpass()
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

 [REDACTED]

In [0]:
OPENAI_API_KEY_TOKEN = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY_TOKEN

 [REDACTED]

In [0]:
%sh
huggingface-cli login --token $HUGGINGFACEHUB_API_TOKEN --add-to-git-credential

Token is valid (permission: fineGrained).
The token `rag_chat_tredence` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (cache).
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `rag_chat_tredence`


In [0]:
# Load Hugging Face models for tasks
# ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
sentiment_pipeline = pipeline("sentiment-analysis", model='distilbert-base-uncased-finetuned-sst-2-english')
translation_pipeline = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr")

Device set to use cpu
/local_disk0/.ephemeral_nfs/envs/pythonEnv-d0076b2f-2da4-44c0-aa59-713021a47f08/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [0]:
from diffusers import DiffusionPipeline
from transformers import BlipProcessor, BlipForConditionalGeneration

image_gen_pipe = DiffusionPipeline.from_pretrained("prompthero/openjourney-v4")
# Load the caption generation model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [0]:
@tool
def generate_image(prompt: str):
    """Generate an image with 3 captions from the given prompt."""
    image = image_gen_pipe(prompt).images[0]
    # Generate captions
    inputs = processor(image, return_tensors="pt")
    outputs = caption_model.generate(**inputs, num_return_sequences=3, num_beams=5)
    captions = [processor.decode(output, skip_special_tokens=True) for output in outputs]
    return image, captions

In [0]:
@tool
def analyze_sentiment(input_text):
    """Analyze the sentiment of the input text."""
    result = sentiment_pipeline(input_text)[0]

    return result['label']

In [0]:
@tool
def extract_named_entities(prompt):
    """Extract named entities from a prompt."""
    # Load the pre-trained spaCy model
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(prompt)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [0]:
@tool
def translate_to_french(prompt: str) -> str:
    """Translate English text to French."""
    translation = translation_pipeline(prompt)
    return translation[0]["translation_text"]

In [0]:
async def create_agent_executor(user_prompt):
    try:

        agent_tools = [analyze_sentiment, extract_named_entities, translate_to_french, generate_image]

        # Use a Hugging Face LLM as the reasoning engine for the agent
        # llm = HuggingFacePipeline(pipeline=pipeline("text-generation", model="Qwen/Qwen2-0.5B", return_full_text=False,))

        llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

        # Get the prompt to use - you can modify this!
        openai_tool_prompt = hub.pull("hwchase17/openai-tools-agent")

        # Construct the ReAct agent
        openai_agent = create_openai_tools_agent(llm, agent_tools, openai_tool_prompt)

        agent_executor = AgentExecutor(
            agent=openai_agent, tools=agent_tools, verbose=True, return_intermediate_steps=False, handle_parsing_errors=True
        )

        response = agent_executor.invoke({"input": user_prompt})

        return response
    except Exception as e:
        print(e)

In [0]:
async def main():
  try:
    while True:
        user_input = input('Message Group 1 AI Service or Type "quit" to exit: ')
        if user_input.lower() == 'quit':
            break

        prompt = user_input
        response = await create_agent_executor(prompt)

        print(response)

    return response

  except Exception as ex:
        print(ex)

In [0]:
s = await main()

Message Group 1 AI Service or Type "quit" to exit:  What is the sentiment of "I love this product, it's amazing!"

/local_disk0/.ephemeral_nfs/envs/pythonEnv-d0076b2f-2da4-44c0-aa59-713021a47f08/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `analyze_sentiment` with `{'input_text': "I love this product, it's amazing!"}`


POSITIVEThe sentiment of the text "I love this product, it's amazing!" is positive.

> Finished chain.
{'input': 'What is the sentiment of "I love this product, it\'s amazing!"', 'output': 'The sentiment of the text "I love this product, it\'s amazing!" is positive.'}


Message Group 1 AI Service or Type "quit" to exit:  Translate "How are you?" to French.

/local_disk0/.ephemeral_nfs/envs/pythonEnv-d0076b2f-2da4-44c0-aa59-713021a47f08/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


...



> Entering new AgentExecutor chain...

Invoking: `generate_image` with `{'prompt': 'A cat playing with Yarn'}`




  0%|          | 0/50 [00:00<?, ?it/s]

(<PIL.Image.Image image mode=RGB size=512x512 at 0x7FD46A215250>, ['a cat sitting next to a ball of yarn', 'a cat is sitting next to a ball of yarn', 'a painting of a cat with a ball of yarn'])I have generated an image of a cat playing with yarn. Here are three captions for the image:
1. A cat sitting next to a ball of yarn
2. A cat is sitting next to a ball of yarn
3. A painting of a cat with a ball of yarn

> Finished chain.
{'input': 'Generate an image and 3 captions for "A cat playing with Yarn"', 'output': 'I have generated an image of a cat playing with yarn. Here are three captions for the image:\n1. A cat sitting next to a ball of yarn\n2. A cat is sitting next to a ball of yarn\n3. A painting of a cat with a ball of yarn'}


Message Group 1 AI Service or Type "quit" to exit:  quit

[Trace(request_id=tr-4629c74e3a1848dcaf1e3749f5fd095a), Trace(request_id=tr-912241b60064497f84bcb0731755860e), Trace(request_id=tr-19745bcc868949ef82f0eaa460c79cf3), Trace(request_id=tr-ea927f8063824dcca3b1fd34295d4438), Trace(request_id=tr-039e73175bf54acf8dc6ed88a28346df), Trace(request_id=tr-ea0e536ea72148f2952a03ac583d7e16)]

In [0]:
# What is the sentiment of "I love this product, it's amazing!"
# Translate "How are you?" to French.
# Extract the entities from the "Andrew is good person to talk"
# Generate an image and 3 captions for "A cat playing with Yarn"
